In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os 
sys.path.append(os.path.abspath('../'))
import torch
from time import time
from tqdm import tqdm
import drjit as dr
import mitsuba as mi
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import clear_output
import torch.nn as nn
import torch.nn.functional as F


from convolutions import *
from utils_fns import *
from utils_general import update_sigma_linear, run_scheduler_step, plt_errors, show_with_error
from optimizations import *
from utils_optim import run_optimization, run_grad_optimization, run_cg_optimization, run_bfgs_optimization
from utils_general import run_scheduler_step
from utils_mitsuba import get_mts_rendering, render_smooth
from read_scenes import create_scene_from_xml

if torch.cuda.is_available():
    device = 'cuda'
    print("is available")
    mi.set_variant('cuda_ad_rgb')

is available


# Setup f(x), g(x)

In [9]:
def f(weights, x):
    '''
    Analytic, for 3d x -> 4d output
    weight matrix with shape (4,3)
    '''
    return weights @ x
    
# def analytical_dfdx(x):
    

In [11]:
x = torch.tensor([0.5, 0.5, 0.5], device=device, requires_grad=True)
weights = torch.rand((4,3), device=device, requires_grad=True)

hess_f = torch.func.hessian(f)
print(weights)
print(hess_f(weights, x))

tensor([[0.5065, 0.5941, 0.4304],
        [0.1630, 0.8501, 0.2484],
        [0.3601, 0.7997, 0.8248],
        [0.0169, 0.4112, 0.7213]], device='cuda:0', requires_grad=True)
tensor([[[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],

      